In [6]:
# Import dependencies
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time 
import requests
import pandas as pd

In [7]:
# Establish a path with Chrome Driver and create browser
executable_path={'executable_path': ChromeDriverManager().install()}
browser=Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430


[WDM] - Driver [C:\Users\renuk\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


In [8]:
# Assign URL and visit with browser
url= "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
browser.visit(url)

In [9]:
# Create an HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')
# Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/) and collect the latest News Title and Paragraph Text. Assign the text to variables.
news_title = soup.find_all("div", class_="content_title")[1].text


In [10]:
# Find and print first paragraph
news_p = soup.find("div", class_="article_teaser_body").text


In [11]:
# Load URL for image and visit using splinter 
image_url= "https://www.jpl.nasa.gov/images?search=&category=Mars"
browser.visit(image_url)

In [12]:
# Find the latest featured image using splinter and click on it
time.sleep(1)
browser.find_by_css('.mb-3').click()
html = browser.html

# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')

In [13]:
# Locate image using splinter, assign to a variable, and save and print complete URL
featured_img= soup.find("img", class_="BaseImage")
featured_img_url= featured_img['data-src']
time.sleep(1)


In [14]:
### Mars Facts

In [15]:
# Visit the Mars Facts webpage (https://space-facts.com/mars/) and assign to variable name
facts_url= "https://space-facts.com/mars/"
browser.visit(facts_url)

In [16]:
# Read table on webpage using Pandas and assign to variable name
mars_facts=pd.read_html(facts_url)

In [17]:
# Create dataframe using scraped data
facts_df = mars_facts[0]

# Rename columns and set 'Category' column as index
facts_df.columns= (['Category', 'Mars Facts'])
mars_facts_clean=facts_df.drop(0)
clean_df=mars_facts_clean.set_index('Category')


In [18]:
# Use Pandas to convert the data to a HTML table string and print as a file.
html_table = facts_df.to_html('mars.html')

In [19]:
# Use Pandas to convert the data to a HTML table string to use in Mongo dictionary.
html_table_mongo = facts_df.to_html()

In [20]:
### Mars Hemispheres

In [21]:
# Visit the USGS Astrogeology site
astrogeo_url= "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(astrogeo_url)

In [22]:
# Create an HTML object
html = browser.html

# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')


In [23]:
# Scrape the USGS Astrogeology site and collect high resolution images for each of Mar's hemispheres. 
# Save both the image url string for the full resolution hemisphere image and the Hemisphere title containing the hemisphere name. 

# Starting with first hemisphere listed, click on hemisphere name
cerb_url=browser.links.find_by_partial_text('Cerberus')
cerb_url.click()

In [24]:
# Sleep browser for one second and create a new HTML object on page
time.sleep(1)
html = browser.html
# Parse Cerberus page HTML with Beautiful Soup
soup_cerb = bs(html, 'html.parser')

# Using BeautifulSoup, locate and save hemisphere image and title 
cerb_photo = soup_cerb.find("a", text="Sample").get("href")
cerb_title = soup_cerb.find("h2", class_="title").text
browser.back()

In [25]:
html = browser.html
# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')

In [26]:
schia_url=browser.links.find_by_partial_text('Schiaparelli')
schia_url.click()

In [27]:
time.sleep(1)
html = browser.html
# Parse HTML with Beautiful Soup
soup_schia = bs(html, 'html.parser')
schia_photo = soup_schia.find("a", text="Sample").get("href")
schia_title = soup_schia.find("h2", class_="title").text
#print(schia_photo)
browser.back()

In [28]:
html = browser.html
# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')

In [29]:
syrtis_url=browser.links.find_by_partial_text('Syrtis')
syrtis_url.click()

In [30]:
time.sleep(1)
html = browser.html
# Parse HTML with Beautiful Soup
soup_syrtis = bs(html, 'html.parser')
syrtis_photo = soup_syrtis.find("a", text="Sample").get("href")
syrtis_title = soup_syrtis.find("h2", class_="title").text
#print(syrtis_photo)
browser.back()

In [31]:
html = browser.html
# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')

In [32]:
valles_url=browser.links.find_by_partial_text('Valles')
valles_url.click()

In [33]:
time.sleep(1)
html = browser.html
# Parse HTML with Beautiful Soup
soup_valles = bs(html, 'html.parser')
valles_photo = soup_valles.find("a", text="Sample").get("href")
valles_title = soup_valles.find("h2", class_="title").text
#print(valles_photo)
browser.back()

In [34]:
browser.quit()

In [35]:
import pymongo

# # Connect to Mongo
# # This needs to be done in python 
mongo_conn='mongodb://localhost:27017'
client=pymongo.MongoClient(mongo_conn)

In [36]:
hemisphere_image_urls=[
    {"title": cerb_title, "img_url": cerb_photo},
    {"title": schia_title, "img_url": schia_photo},
    {"title": syrtis_title, "img_url": syrtis_photo},
    {"title": valles_title, "img_url": valles_photo}
]

In [37]:
#Store table and headlines in dictionary
mars_facts= {
    'Headline': news_title,
    'News': news_p,
    'Featured_Image': featured_img_url,
    'Table': html_table_mongo,
    'Hemisphere_Images': hemisphere_image_urls
    }
# print(mars_facts)
# Import to MongoDB
client.mars_db.mars.insert_one(mars_facts)

In [41]:
# conn = 'mongodb://localhost:27017'
# import pymongo
# PyMongo constructor - this is a class, as noted by the capital letters
# client = pymongo.MongoClient(conn)
# db=client.MarsDB
# response= requests.get(url)
# facts_df.drop("index")
# facts_df.set_index=(['Category'])
#Store hemisphere labels and images in a dictionary. Use a Python dictionary to store the data using the keys `img_url` and `title`.
# hemisphere_image_urls=[
#     {"title": cerb_title, "img_url": cerb_photo},
#     {"title": schia_title, "img_url": schia_photo},
#     {"title": syrtis_title, "img_url": syrtis_photo},
#     {"title": valles_title, "img_url": valles_photo}
  
# ]
# #print(hemisphere_image_urls)

# Import to MongoDB
# client.mars_db.mars.insert_one(hemisphere_image_urls)
# Put all 4 in a loop, will click into first one, scrape the website, grab text and full size image, store those
#i n a dictionary, then goes to next link do same thing, etc. 4 